In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, UpSampling2D, BatchNormalization
from keras.metrics import RootMeanSquaredError
from keras.optimizers import AdamW
from keras import backend as K

In [23]:
X_train = np.load('../y_smp_train.npy')
y_train = np.load('../pars_smp_train.npy')
X_test = np.load('../y_smp_test.npy')

In [14]:
X_train.shape, y_train.shape

((100000, 200, 3), (100000, 15, 1))

In [2]:
def quantile_loss_25(y_true, y_pred):
    error = y_true - y_pred
    loss = K.mean(K.maximum(0.25 * error, (0.25 - 1) * error))
    return loss

In [5]:
time_steps = 200
num_features = 3

In [6]:
model = Sequential()
model.add(Conv2D(16, (3,3), activation='relu', input_shape=(time_steps, num_features, 1), padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(32, (12,6), activation='relu', strides=(6, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Conv2D(16, (3,3), activation='relu', strides=(2, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dense(15))

opt = AdamW(learning_rate=0.0001)
model.compile(loss=quantile_loss_25, optimizer=opt, metrics=[RootMeanSquaredError()])

In [7]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 200, 3, 16)        160       
                                                                 
 up_sampling2d_2 (UpSamplin  (None, 400, 6, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 400, 6, 32)        4640      
                                                                 
 up_sampling2d_3 (UpSamplin  (None, 800, 12, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 132, 7, 32)        73760     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 66, 4, 32)        

In [18]:
model.fit(X_train, y_train, batch_size=128, epochs=25)

276/782 [=========>....................] - ETA: 4s - loss: 0.0619 - root_mean_squared_error: 0.3357

Failed to deserialize variable 'model'. Run the following code to delete it:
  del_datasphere_variables('model')
Traceback (most recent call last):
  File "/kernel/lib/python3.10/site-packages/ml_kernel/state/state_protocol.py", line 283, in _load_component
    value = unpickler.load()
  File "/kernel/lib/python3.10/site-packages/ipystate/impl/dispatch/tensorflow.py", line 74, in _load_model
    restored_model = tf.keras.models.load_model(model_path)
  File "/usr/local/lib/python3.10/dist-packages/keras/saving/saving_api.py", line 212, in load_model
    return legacy_sm_saving_lib.load_model(
  File "/usr/local/lib/python3.10/dist-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.10/dist-packages/keras/saving/legacy/serialization.py", line 543, in deserialize_keras_object
    raise ValueError(
ValueError: Unknown loss function: 'quantile_loss_25'. Please ensure you are using a `keras.util

Epoch 1/25
782/782 [==============================] - 15s 9ms/step - loss: 0.1612 - root_mean_squared_error: 0.8700
Epoch 2/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0868 - root_mean_squared_error: 0.4687
Epoch 3/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0788 - root_mean_squared_error: 0.4214
Epoch 4/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0748 - root_mean_squared_error: 0.3971
Epoch 5/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0718 - root_mean_squared_error: 0.3809
Epoch 6/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0689 - root_mean_squared_error: 0.3648
Epoch 7/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0669 - root_mean_squared_error: 0.3555
Epoch 8/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0653 - root_mean_squared_error: 0.3489
Epoch 9/25
782/782 [==============================] - 7s 9ms/step - loss: 0.064

In [19]:
pred = model.predict(X_test)

Failed to deserialize variable 'model'. Run the following code to delete it:
  del_datasphere_variables('model')
Traceback (most recent call last):
  File "/kernel/lib/python3.10/site-packages/ml_kernel/state/state_protocol.py", line 283, in _load_component
    value = unpickler.load()
  File "/kernel/lib/python3.10/site-packages/ipystate/impl/dispatch/tensorflow.py", line 74, in _load_model
    restored_model = tf.keras.models.load_model(model_path)
  File "/usr/local/lib/python3.10/dist-packages/keras/saving/saving_api.py", line 212, in load_model
    return legacy_sm_saving_lib.load_model(
  File "/usr/local/lib/python3.10/dist-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.10/dist-packages/keras/saving/legacy/serialization.py", line 543, in deserialize_keras_object
    raise ValueError(
ValueError: Unknown loss function: 'quantile_loss_25'. Please ensure you are using a `keras.util

3125/3125 [==============================] - 4s 1ms/step


In [20]:
np.save('test_pred_025', pred)

In [21]:
model.save("model_025.h5")

Failed to deserialize variable 'model'. Run the following code to delete it:
  del_datasphere_variables('model')
Traceback (most recent call last):
  File "/kernel/lib/python3.10/site-packages/ml_kernel/state/state_protocol.py", line 283, in _load_component
    value = unpickler.load()
  File "/kernel/lib/python3.10/site-packages/ipystate/impl/dispatch/tensorflow.py", line 74, in _load_model
    restored_model = tf.keras.models.load_model(model_path)
  File "/usr/local/lib/python3.10/dist-packages/keras/saving/saving_api.py", line 212, in load_model
    return legacy_sm_saving_lib.load_model(
  File "/usr/local/lib/python3.10/dist-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.10/dist-packages/keras/saving/legacy/serialization.py", line 543, in deserialize_keras_object
    raise ValueError(
ValueError: Unknown loss function: 'quantile_loss_25'. Please ensure you are using a `keras.util